In [ ]:
import pickle
import json
from scipy.interpolate import interp1d
import os
import numpy as np
import pandas as pd

In [ ]:
def compute_injection_credible_level (x, prob_x, x_inj):    

    credible_level = np.sum(prob_x[x<x_inj])/np.sum(prob_x)
    credible_level = np.array(credible_level)
    credible_level[credible_level>0.5] = 1 - credible_level[credible_level>0.5]
    credible_level *= 2
    return credible_level

In [ ]:
def np_to_json(o):
    if isinstance(o, np.ndarray):
        return o.item() if o.shape == () else o.tolist()
    if isinstance(o, (np.floating,)): return float(o)
    if isinstance(o, (np.integer,)):  return int(o)
    if isinstance(o, (np.bool_,)):    return bool(o)
    return o  # dict, list, str, float, int, None

In [ ]:
dict_of_dicts = {}
param_list_short = ['chirp_mass','mass_ratio', 'a_1', 'a_2', 'theta_jn', 'luminosity_distance']
param_labels = [r"$\mathcal{M}_c$", r"$q$", r"$a_1$", r"$a_2$", r"$\theta_{jn}$", r"$d_L$"]

In [ ]:
# Iterate through all parameters
for k, param in enumerate(param_list_short):
    credible_level_dict = {}
    
    # Process 'Unclean' data
    credible_levels = []
    for i in range(1, 128):
        with open(f"posteriors/unclean/results_{i}.pkl", 'rb') as file:
            loaded_data = pickle.load(file)
        pdf_interp = interp1d(loaded_data[f'grid_{param}'], loaded_data[f'pdf_{param}'])
        cl = compute_injection_credible_level(pdf_interp.x, pdf_interp.y, loaded_data[f'inj_{param}'])    
        credible_levels.append(cl)
    credible_level_dict['Unclean'] = credible_levels

    # Process 'DeepClean' data
    credible_levels = []
    for i in range(1, 128):
        with open(f"posteriors/deepclean/results_{i}.pkl", 'rb') as file:
            loaded_data = pickle.load(file)
        pdf_interp = interp1d(loaded_data[f'grid_{param}'], loaded_data[f'pdf_{param}'])
        cl = compute_injection_credible_level(pdf_interp.x, pdf_interp.y, loaded_data[f'inj_{param}'])    
        credible_levels.append(cl)
    credible_level_dict['DeepClean'] = credible_levels

    # Add the credible level dictionary to the top-level dictionary
    dict_of_dicts[param] = credible_level_dict


In [ ]:
with open("credible_levels.json", "w") as f:
    json.dump(dict_of_dicts, f, indent=2, default=np_to_json)